In [17]:
# Importing necessary models
import warnings
warnings.filterwarnings('ignore')

import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
import time
%matplotlib inline
from bs4 import BeautifulSoup as bs
import requests
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,.4f}'.format

init_notebook_mode(connected=True)

import os
main_dir = os.getcwd()

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')

from yahoo_query import *

os.chdir('C:\\Users\\Fang\\Downloads')

In [18]:
start_date = dt.datetime(1950,1,3)

try:
    
    spx = yahoo_query('^GSPC', start_date)
    spx.hist_prices_query()
    spx = spx.hist_prices
except:
    spx = pd.read_csv('^GSPC.csv', index_col = 0)

In [125]:
from scipy.optimize import curve_fit


def generate_sim(n, sim_days, sim_nums):
    empirical_dist = np.log(spx[['Close']]/spx[['Close']].shift(n)).sort_values('Close')
    empirical_dist['cumprob'] = np.arange(len(empirical_dist))/len(empirical_dist)


    def sigmoid(x, x0, k):
        y = 1 / (1 + np.exp(-k*(x-x0)))
        return y

    popt, pcov = curve_fit(sigmoid, empirical_dist[empirical_dist.Close < 0].Close, empirical_dist[empirical_dist.Close < 0].cumprob)

    #print(popt)

    x = np.linspace(empirical_dist.Close.min(), empirical_dist.Close.max(), len(empirical_dist.Close))
    y = sigmoid(x, *popt)

    # fig = plt.figure(figsize=(20,20))

    # ax = fig.add_subplot(111)

    # ax.plot(empirical_dist.Close, empirical_dist.cumprob, 'o', label='data')
    # ax.plot(x,y, label='fit')
    # #pylab.ylim(0, 1.05)
    # ax.legend(loc='best')

    def sigmoid_inv(y, x0, k):
        x = -(np.log((1-y)/y)/k) + x0
        return x
    
    return sigmoid_inv(np.random.random(size = (sim_nums, sim_days)), *popt)

In [134]:
sim_nums = 100000

sim = generate_sim(1, 300, sim_nums)

strike = 2700
curr_spx = spx.reset_index().loc[len(spx) - 1, 'Close']

sum(np.maximum(strike - curr_spx*np.exp(np.sum(sim, axis = 1)),0))/sim_nums

sim

array([[ 0.00746006, -0.00456509,  0.01135826, ...,  0.00066627,
        -0.02481364,  0.00771151],
       [-0.00142143,  0.00389354,  0.00099432, ...,  0.00077239,
         0.01619721, -0.00091859],
       [ 0.00641565,  0.00534489, -0.00799598, ...,  0.00169888,
         0.00552178, -0.01160711],
       ..., 
       [ 0.00576035, -0.00539668, -0.00285539, ...,  0.00178283,
        -0.00512791,  0.00475185],
       [-0.00532137, -0.01321605,  0.00461922, ...,  0.00216451,
         0.0018762 , -0.0036041 ],
       [ 0.00907376,  0.00140669, -0.00091813, ...,  0.00368678,
        -0.00434973,  0.00778291]])